In [41]:
from transformers import pipeline

In [42]:
classifier = pipeline(
    "sentiment-analysis",
    model = "distilbert-base-uncased-finetuned-sst-2-english"
)

Loading weights: 100%|██████████| 104/104 [00:00<00:00, 387.32it/s, Materializing param=pre_classifier.weight]                                  


In [43]:
text = "I absolutely love this movie! Great acting."
result = classifier(text)
print(result)

[{'label': 'POSITIVE', 'score': 0.9998805522918701}]


In [44]:
text = "Lovely cant you just do simple thing!!!"
result = classifier(text)
print(result)

[{'label': 'POSITIVE', 'score': 0.9993396401405334}]


## NER

In [45]:
ner = pipeline(
    task="ner",
    model="dslim/bert-base-NER",
    aggregation_strategy="simple"
)

Loading weights: 100%|██████████| 199/199 [00:00<00:00, 373.75it/s, Materializing param=classifier.weight]                                      
BertForTokenClassification LOAD REPORT from: dslim/bert-base-NER
Key                      | Status     |  | 
-------------------------+------------+--+-
bert.pooler.dense.bias   | UNEXPECTED |  | 
bert.pooler.dense.weight | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [46]:
text = "Apple CEO Tim Cook announced new products at Cupertino."
entities = ner(text)
for entity in entities:
    print(
        f"{entity['word']} → {entity['entity_group']} "
        f"(confidence: {entity['score']:.2f})"
    )

Apple → ORG (confidence: 1.00)
Tim Cook → PER (confidence: 1.00)
Cupertino → LOC (confidence: 0.97)


In [47]:
text = "Apple CEO Tim Cook announced new products at Cupertino."
entities = ner(text)
for entity in entities:
    print(
        f"{entity['word']} → {entity['entity_group']} "
        f"(confidence: {entity['score']:.2f})"
    )

Apple → ORG (confidence: 1.00)
Tim Cook → PER (confidence: 1.00)
Cupertino → LOC (confidence: 0.97)


In [48]:
text = "Ganesh Bhel owner is Raju has famous food."
entities = ner(text)
for entity in entities:
    print(
        f"{entity['word']} → {entity['entity_group']} "
        f"(confidence: {entity['score']:.2f})")
    

G → PER (confidence: 0.60)
##anesh Bhel → ORG (confidence: 0.52)
Raj → PER (confidence: 0.63)
##u → PER (confidence: 0.78)


## text generation

In [49]:
from transformers import pipeline

In [50]:
generator = pipeline(
    "text-generation",
    model = "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    max_new_tokens = 150,
    temperature = 0.75,
    do_sample=True,      # Required to use top_p
    top_p=0.95,
)

Loading weights: 100%|██████████| 201/201 [00:00<00:00, 372.01it/s, Materializing param=model.norm.weight]                              
Passing `generation_config` together with generation-related arguments=({'do_sample', 'top_p', 'max_new_tokens', 'temperature'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.


In [51]:
result = generator("Explain deep learning in 2 lines:")

Both `max_new_tokens` (=150) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [52]:
output_text = result[0]["generated_text"]


# Print the generated text
print(output_text)


Explain deep learning in 2 lines:

1. Convolutional Neural Networks (CNNs) are a family of deep learning algorithms that use convolutional layers to extract features from images.

2. Recurrent Neural Networks (RNNs) are another family of deep learning algorithms that use recurrent layers to generate sequences or patterns.

3. Deep Learning can be understood as a set of techniques that can be applied to neural networks to improve their performance in a variety of applications.

4. Deep Learning is a broad field that covers a wide range of techniques, including:

1. Convolutional Neural Networks (CNNs)
2. Recurrent Neural Networks (RNNs)
3


## question answer

In [53]:
qa = pipeline(
    "question-answering",
    model="deepset/roberta-base-squad2"
    #doc_stride when the model has huge context then it adds the overlapping of the text
)

Loading weights: 100%|██████████| 199/199 [00:00<00:00, 403.24it/s, Materializing param=roberta.encoder.layer.11.output.dense.weight]              
RobertaForQuestionAnswering LOAD REPORT from: deepset/roberta-base-squad2
Key                             | Status     |  | 
--------------------------------+------------+--+-
roberta.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [54]:
context = (
    "Fanatic won Masters and regional championship in year 2024"
    "They won against loud with 3-11 comeback which was the greatest comeback of all time in history"
)

question = "When did fanatic won championship?"

answer = qa(
    question=question,
    context=context
)
print(f"Answer: {answer['answer']}")

Answer: 2024


In [55]:
question = "how did fanatic won against loud?"

answer = qa(
    question=question,
    context=context
)
print(f"Answer: {answer['answer']}")

Answer: 3-11 comeback


## BERT

In [1]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification,Trainer
from datasets import load_dataset

d:\Training\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset(
    'csv',
    data_files="Complaints.csv"
)

Generating train split: 11 examples [00:00, 199.11 examples/s]


In [3]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [4]:
train_dataset = dataset.map(
    lambda x:tokenizer(
        x["text"],
        padding = 'max_length',
        truncation = True
    ),
    batched=True
)

Map: 100%|██████████| 11/11 [00:00<00:00, 259.04 examples/s]


In [5]:
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=3
)

Loading weights: 100%|██████████| 199/199 [00:00<00:00, 332.24it/s, Materializing param=bert.pooler.dense.weight]                               
BertForSequenceClassification LOAD REPORT from: bert-base-uncased
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
classifier.bias                            | MISSING    | 
classifier.weight                          | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those pa

In [6]:
trainer = Trainer(
    model = model,
    train_dataset=train_dataset["train"]
)

In [7]:
from transformers import pipeline
model.config.id2label = {
    0: "NEGATIVE",
    1: "NEUTRAL",
    2: "POSITIVE"
}

model.config.label2id = {
    "NEGATIVE": 0,
    "NEUTRAL": 1,
    "POSITIVE": 2
}

In [8]:
classifier = pipeline(
    task="text-classification",
    model=model,
    tokenizer=tokenizer,
    return_all_scores=True
)

In [9]:


# ------------------------------------------------------------
# STEP 3: TEST DATA (REAL-WORLD + EDGE CASES)
# ------------------------------------------------------------
test_sentences = [
    "The service was extremely bad and disappointing.",
    "The experience was okay, nothing special.",
    "Absolutely loved the customer support!",
    "Not bad, but could be better.",
    "Worst experience ever.",
    "I am happy with the service.",
    "",                         # empty input
    "ok",                       # very short input
    "asdfghjkl",                # meaningless input
    "The product was not bad",  # negation
    "I don't hate the service"  # double negation
]


# ------------------------------------------------------------
# STEP 4: RUN PREDICTIONS AND DISPLAY RESULTS
# ------------------------------------------------------------
print("\n================ MODEL TEST RESULTS ================\n")

for text in test_sentences:
    # ✅ CHANGED (kept): Normalize pipeline outputs to a list-of-dicts to avoid TypeError
    outputs = classifier(text, top_k=None)
    if isinstance(outputs, dict):
        results = [outputs]                 # single dict
    elif outputs and isinstance(outputs[0], list):
        results = outputs[0]                # [[{label, score}, ...]] → take first
    elif outputs and isinstance(outputs[0], dict):
        results = outputs                   # [{label, score}]
    else:
        results = []                        # fallback empty

    # Select label with highest confidence
    best_prediction = max(results, key=lambda x: x["score"]) if results else {"label": "N/A", "score": 0.0}

    print(f"Input Text: '{text}'")
    for r in results:
        # ✅ CHANGED: replace HTML-escaped '&gt;' with the correct '>' alignment
        print(f"  {r['label']:>8} → {r['score']:.3f}")

    print(f"  ▶ Final Prediction: {best_prediction['label']} "
          f"({best_prediction['score']:.2f})")
    print("-" * 55)


print("\n✅ Model testing completed successfully.\n")


================ MODEL TEST RESULTS ================

Input Text: 'The service was extremely bad and disappointing.'
  NEGATIVE → 0.496
   NEUTRAL → 0.282
  POSITIVE → 0.222
  ▶ Final Prediction: NEGATIVE (0.50)
-------------------------------------------------------
Input Text: 'The experience was okay, nothing special.'
  NEGATIVE → 0.401
   NEUTRAL → 0.302
  POSITIVE → 0.297
  ▶ Final Prediction: NEGATIVE (0.40)
-------------------------------------------------------
Input Text: 'Absolutely loved the customer support!'
  NEGATIVE → 0.351
   NEUTRAL → 0.330
  POSITIVE → 0.319
  ▶ Final Prediction: NEGATIVE (0.35)
-------------------------------------------------------
Input Text: 'Not bad, but could be better.'
  NEGATIVE → 0.447
   NEUTRAL → 0.290
  POSITIVE → 0.263
  ▶ Final Prediction: NEGATIVE (0.45)
-------------------------------------------------------
Input Text: 'Worst experience ever.'
  NEGATIVE → 0.542
   NEUTRAL → 0.265
  POSITIVE → 0.193
  ▶ Final Prediction: NEGATIVE 